In [1]:
import re
import subprocess
from pathlib import Path
import numpy as np
import plotly.graph_objects as go
import pandas as pd

In [8]:
p = Path('logs')
directories = [x.name for x in p.iterdir() if x.is_dir()]
lines_dict = {}

In [9]:
def tail(f, n, offset=0):
    proc = subprocess.Popen(['tail', '-n', str(n + offset), f], stdout=subprocess.PIPE)
    lines = proc.stdout.readlines()
    return lines[:offset]

def return_result(f):
    result = [x.decode("utf-8") for x in tail(f, 7, 2)]
    result = [re.findall('\d+', result[0])[0]] + re.findall('\d+', result[1])
    result = [float(x) for x in result] # length of 7, [deg, wt pz/1-pz, 0.1, 0.3, 0.5, 0.7, 1.0]
    return result

In [10]:
for dir in directories:
    [N,K,l,s,n,exact] = [''.join(x for x in r if x.isdigit()) for r in dir.split('_')[:-1]]
    con = dir.split('_')[-1]
    exp_name = 'N='+N+',K='+K+',l='+l+',n='+n+',exact='+exact+','+con
    print(exp_name)
    q = p / dir
    files = list(q.glob('*.log'))
    pzs = [''.join(x for x in f.name if x.isdigit()) for f in files]
    pzs = np.array([float(pz) for pz in pzs]) / 100.0
    results = [return_result(f) for f in files]
    deg_err = np.array([x[0] for x in results]) / float(n)
    true_weight_err = np.array([x[1] for x in results]) / float(n)
    # sort so that the line will be smooth
    order = np.argsort(pzs)
    pzs = pzs[order]
    deg_err = deg_err[order]
    true_weight_err = true_weight_err[order]
    exp_name_deg = exp_name + ',deg'
    exp_name_wt = exp_name + ',wt'
    if exp_name_deg not in lines_dict:
        lines_dict[exp_name_deg]= (pzs, [deg_err])
        lines_dict[exp_name_wt] = (pzs, [true_weight_err])
    else:
        lines_dict[exp_name_deg][1].append(deg_err)
        lines_dict[exp_name_wt][1].append(true_weight_err)

N=1024,K=513,l=128,n=10000,exact=0,RM
N=256,K=129,l=128,n=10000,exact=0,RM
N=256,K=129,l=128,n=10000,exact=0,PW
N=64,K=33,l=128,n=10000,exact=0,RM
N=1024,K=513,l=128,n=10000,exact=0,HPW
N=1024,K=513,l=128,n=10000,exact=0,PW
N=128,K=65,l=128,n=10000,exact=0,RM
N=128,K=65,l=128,n=10000,exact=0,PW
N=512,K=257,l=128,n=10000,exact=0,RM
N=512,K=257,l=128,n=10000,exact=0,PW
N=64,K=33,l=128,n=10000,exact=0,PW
N=512,K=257,l=128,n=10000,exact=0,HPW
N=256,K=129,l=128,n=10000,exact=0,HPW
N=128,K=65,l=128,n=10000,exact=0,HPW
N=64,K=33,l=128,n=10000,exact=0,HPW
N=1024,K=513,l=128,n=10000,exact=0,PW
N=1024,K=513,l=128,n=10000,exact=0,HPW
N=1024,K=513,l=128,n=10000,exact=0,PW
N=1024,K=513,l=128,n=10000,exact=0,HPW
N=1024,K=513,l=128,n=10000,exact=0,PW
N=1024,K=513,l=128,n=10000,exact=0,HPW
N=1024,K=513,l=128,n=10000,exact=0,PW
N=1024,K=513,l=128,n=10000,exact=0,PW
N=1024,K=513,l=128,n=10000,exact=0,HPW
N=1024,K=513,l=128,n=10000,exact=0,HPW
N=1024,K=513,l=128,n=10000,exact=0,PW
N=1024,K=513,l=128,n=10

In [11]:
data_dict = {}
for key, value in lines_dict.items():
    mean = np.mean(value[1], axis=0)
    var = np.std(value[1], axis=0)
    data_dict[key] = (value[0], mean, var)
    

In [12]:
fig = go.Figure()

for x,y in data_dict.items():
    if ('RM' in x) or ('HPW' in x) or ('1024' not in x):
        continue
    fig.add_scatter(x=y[0], y=y[1], error_y=dict(type='data', array=y[2], visible=True), name=x)
    # fig.add_trace(go.Scatter(x=y[0], y=y[1], mode='lines+markers', name=x))

fig.show()

In [13]:
fig = go.Figure()

for x,y in data_dict.items():
    if ('PW' in x):
        fig.add_scatter(x=y[0], y=y[1], error_y=dict(type='data', array=y[2], visible=True), name=x)

fig.show()
fig.write_html("HPW_vs_PW.html")

In [9]:
fig = go.Figure()

for x,y in data_dict.items():
    if ('PW' in x):
        continue
    fig.add_trace(go.Scatter(x=y[0], y=y[1], mode='lines+markers', name=x))

fig.show()